In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import os

# Load environment variables from the .env file
load_dotenv()

# Get the API key from the environment variable

open_api_key = os.getenv('OPENAI_API_KEY')

# Initialize OpenAI client

openai_client = OpenAI(api_key = open_api_key)



In [2]:
import importlib
import modules.pdf_extraction
importlib.reload(modules.pdf_extraction)
from modules.pdf_extraction import extract_text_from_pages, select_pdf_file



ModuleNotFoundError: No module named 'modules'

In [38]:

pdf_path = select_pdf_file()
# input page no
page_no = 0

extracted_text = extract_text_from_pages(pdf_path, pages=page_no)


In [40]:
import pymupdf 
## 111
import base64
import requests

import importlib
import modules.llm
import modules.pdf_extraction
importlib.reload(modules.llm)
importlib.reload(modules.pdf_extraction)

from modules.llm import pattern_description_llm, table_extraction
from modules.pdf_extraction import get_page_pixel_data


base64_image = get_page_pixel_data(pdf_path=pdf_path, page_no=page_no, 
                    dpi = 500, image_type = 'png')

# perhaps let it run twice to make super sure. if the numbers do not match run again and pick the max. If it still does not work then just pick one
pattern_desc_from_image = pattern_description_llm(text_input = extracted_text, model='gpt-4o',  base64_image = base64_image, open_api_key=open_api_key)

pattern_desc_from_image

# Function to process each row in parallel
# open ai does not seem to have structed output for this image input version.
# Ill pass it as text to the next llm# 111

'1. Number of Tables on the Page: 1\n\n2. Table Headers: Organic Items Price/kg Quantity(kg) Subtotal\n\n3. Rules for Parsing Tables and Headers:\n   - Look for sequences of words followed by consistent data types (e.g., words followed by numbers), indicating tabular data.\n   - Identify headers as lines with multiple columns where text typically appears capitalized or bold.\n   - Table headers often contain key business terms or product descriptions.\n   - Look for repeated patterns, like price followed by a unit and quantity, as an indicator of rows in a table.'

In [41]:
# Extract the respective variable outputs


import re

text = pattern_desc_from_image
# 1) Extract the Number of Tables
match_num_tables = re.search(r'Number of Tables on the Page:\s*(\d+)', text)
if match_num_tables:
    num_tables = int(match_num_tables.group(1))
else:
    num_tables = None  # or handle error

# 2) Extract the Table Headers
#    We'll grab everything after "Table Headers:" until the line "3. Rules" begins
match_headers = re.search(r'Table Headers:\s*(.*?)\s*\n\s*3\.', text, re.DOTALL)
if match_headers:
    headers_text = match_headers.group(1)
    # Split on '||' to get individual headers
    table_headers = [h.strip() for h in headers_text.split('||')]
else:
    table_headers = []

# 3) Extract the Rules for Parsing Tables and Headers
#    We'll capture everything after "3. Rules for Parsing Tables and Headers:" until the end
match_rules = re.search(r'3\. Rules for Parsing Tables and Headers:\s*(.*)', text, re.DOTALL)
if match_rules:
    parsing_rules = match_rules.group(1).strip()
else:
    parsing_rules = ""

# # Print or use these variables as needed
# print("Number of Tables:", num_tables)
# print("Table Headers:", table_headers)
# print("Rules for Parsing Tables and Headers:\n", parsing_rules)


In [42]:
table_headers

['Organic Items Price/kg Quantity(kg) Subtotal']

In [43]:
# Numerical columns as an additional feature

user_text='Extract all the data from the table also create a column with which contains the name of the farm for each row'

import modules.llm
importlib.reload(modules.llm)
from modules.llm import vision_column_llm_parser
# use user input to determine table of interst
# variable_col_data = vision_column_llm_parser(user_text='Extract all data from the table i also want a column which tells me when the bend radius is above 1000',
#                                     table_to_target= table_headers[0],
#                                     base64_image=base64_image,
#                                     open_api_key=open_api_key)

variable_col_data = vision_column_llm_parser(user_text=user_text,
                                   text_input= extracted_text,
                                    table_to_target= headers_text,
                                    base64_image=base64_image,
                                    open_api_key=open_api_key)
variable_col_data

'``````````````````````\nindex: [0], table_header or descriptor: [Organic Items Price/kg Quantity(kg) Subtotal], column_names: [organic_items, price_kg, quantity_kg, subtotal, farm_name]\n``````````````````````'

In [44]:
# import re

# # Your text
# text =variable_col_data

# pattern = r'index:\s*\[(\d+)\].*?column_names:\s*\[(.*?)\]'
# matches = re.findall(pattern, text)

# results = []
# for index_str, columns_str in matches:
#     index_value = int(index_str)
    
#     # 1) Strip any extra whitespace
#     # 2) Then remove leading/trailing brackets (e.g. "[..." or "...]")
#     columns_str = columns_str.strip().strip("[]")
    
#     # Now split on commas to get a proper list
#     columns_list = [col.strip() for col in columns_str.split(',')]
    
#     # Fetch the correct table_header using index
#     header = table_headers[index_value]
    
#     results.append({
#         "index": index_value,
#         "table_header": header,
#         "column_names": columns_list
#     })




In [45]:
#type(results[0]['column_names'])

In [46]:

text = variable_col_data
pattern = r'index:\s*\[(\d+)\].*?column_names:\s*\[(.*?)\]'

matches = re.findall(pattern, text)

results = []
for index_str, columns_str in matches:
    index_value = int(index_str)
    columns_list = [col.strip() for col in columns_str.split(',')]
    
    # Use the table_headers list to fetch the header based on index
    header = table_headers[index_value]
    
    results.append({
        "index": index_value,
        "table_header": header,
        "column_names": columns_list
    })

In [47]:

# import importlib
# import modules.llm
# importlib.reload(modules.llm)
# from modules.llm import variable_extractor_llm

# var_list =  variable_extractor_llm(user_text= 'return every column in the table',
#                                     text_data=extracted_text, 
#                                     target_table= table_headers[2],
#                                     pattern =parsing_rules,
#                                     openai_client=openai_client, 
#                                     model='gpt-4o') # gpt-4o
# var_list



In [48]:
table_index = 0
table_headers[0]

'Organic Items Price/kg Quantity(kg) Subtotal'

In [49]:
input_fields = sorted(set(results[table_index]['column_names'])) # in case of dups
input_fields

['farm_name', 'organic_items', 'price_kg', 'quantity_kg', 'subtotal']

In [50]:

# import base64

# def load_image(inputs: dict) -> dict:
#     """Load image from file and encode it as base64."""
#     image_path = inputs["image_path"]
  
#     def encode_image(image_path):
#         with open(image_path, "rb") as image_file:
#             return base64.b64encode(image_file.read()).decode('utf-8')
#     image_base64 = encode_image(image_path)
#     return {"image": image_base64}


# from langchain.chains import TransformChain

# load_image_chain = TransformChain(
#     input_variables=["image_path"],
#     output_variables=["image"],
#     transform=load_image
# )

# from langchain_core.pydantic_v1 import BaseModel, Field

# class ImageInformation(BaseModel):
#  """Information about an image."""
#  column_0303: list[str] = Field(description="Column name called 0303")
#  bend_radius_mm: list[str] = Field(description="Column   name called bend_radius_mm")
#  burst_pressure_bar: list[str] = Field(description="Column name called burst_pressure_bar")
#  id: list[str] = Field(description="Column name called id")
#  max_pressure_bar: list[str] = Field(description="Column name called max_pressure_bar")
#  od_mm: list[str] = Field(description="Column name called od_mm")




# from langchain.chains import TransformChain
# from langchain_core.messages import HumanMessage
# from langchain_openai import ChatOpenAI
# from langchain import globals
# from langchain_core.runnables import chain
# from langchain_core.output_parsers import JsonOutputParser


# parser = JsonOutputParser(pydantic_object=ImageInformation)

# # Set verbose
# globals.set_debug(True)

# @chain
# def image_model(inputs: dict) -> str | list[str] | dict:
#  """Invoke model with image and prompt."""
#  model = ChatOpenAI(temperature=0.8, model="gpt-4o", max_tokens=3000)
#  msg = model.invoke(
#              [HumanMessage(
#              content=[
#              {"type": "text", "text": inputs["prompt"]},
#              {"type": "text", "text": parser.get_format_instructions()},
#              {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{inputs['image']}"}},
#              ])]
#              )
#  return msg.content



# # vision_prompt =   f"""  

# # You are a skilled data extraction and parsing specialist with expertise in analysing PDF text and image documents and extracting structured information from tables.

# #           You will be provided with:

# #           * A users request. 
# #           * Table headers or descriptors
# #           * An image of the PDF page for reference.
# #           * All text data on the PDF page. 

       
# #         Your job is to read a user's request, the image of the page and the tex document. 
# #         You are to identify and extract the column names of each table specified in line with the user's request. 
# #         If there are multiple tables they will be sepreated with " || " .
# #         Column names refer to the labels assigned to each column with populated values. Columns names may be text, numerical, alphanumerial and could sometimes contian special characters.
# #         The user request may sometimes imply the extraction of select column names or generating new ones. e.g header column name
# #         Return these column names in a list with the corresponding table header and description and its index in the list. 
# #         For a giving column name ensure the spaces within it are replaced with an under score for example (if the variable is "pack quantity" ensure you return "pack_qunatity") and ensure it is lower cased
        
# #         Example
        
# #         ``````````````````````
# #         User's request: I want to extract all data in the columns as well as the header which is ontop of the table
# #         ``````````````````````

# #         ``````````````````````
# #         Table to extract the column names from: [Metric Male 24 Cone Seat,S Series, DIN 3865 || Standpipe,S Series]
# #         ``````````````````````
        
# #         ``````````````````````
# #         Text document: '\n\n--- Page 1 ---\n\n21\n1.01  Hydraulic Hose & Inserts\nHydraulics\n1\n Metric Heavy Series\n Hexavalent chromium free plating\nMetric Male 24¡ Cone Seat, \nS Series, DIN 3865\n0309\nThread\nTo Suit\nHose ID\nSize\nHMMS03-M16CF\nM16 x 1.5\n3/16”\n8S\nHMMS04-M14CF\nM14 x 1.5\n1/4”\n6S\nHMMS04-M16CF\nM16 x 1.5\n1/4”\n8S\nHMMS04-M18CF\nM18 x 1.5\n1/4”\n10S\nHMMS05-M18CF\nM18 x 1.5\n5/16”\n10S\nHMMS05-M20CF\nM20 x 1.5\n5/16”\n12S\nHMMS06-M18CF\nM18 x 1.5\n3/8”\n10S\nHMMS06-M20CF\nM20 x 1.5\n3/8”\n12S\nHMMS06-M22CF\nM22 x 1.5\n3/8”\n14S\nHMMS06-M24CF\nM24 x 1.5\n3/8”\n16S\nHMMS08-M24CF\nM24 x 1.5\n1/2”\n16S\nHMMS10-M30CF\nM30 x 2\n5/8”\n20S\nHMMS12-M30CF\nM30 x 2\n3/4”\n20S\nHMMS12-M36CF\nM36 x 2\n3/4”\n25S\nHMMS16-M36CF\nM36 x 1.5\n1”\n25S\nHMMS16-M42CF\nM42 x 2\n1”\n30S\nHMMS20-M42CF\nM42 x 2\n1.1/4”\n30S\nHMMS20-M52CF\nM52 x 2\n1.1/4”\n38S\nHMMS24-M52CF\nM52 x 2\n1.1/2”\n38S\nMetric Female 24¡ Cone Seat, \nS Series, DIN 3865\n0309\nThread\nTo Suit\nHose ID\nSize\nHMFS03-M16CF\nM16 x 1.5\n3/16”\n8S\nHMFS04-M14CF\nM14 x 1.5\n1/4”\n6S\nHMFS04-M16CF\nM16 x 1.5\n1/4”\n8S\nHMFS04-M18CF\nM18 x 1.5\n1/4”\n10S\nHMFS05-M18CF\nM18 x 1.5\n5/16”\n10S\nHMFS05-M20CF\nM20 x 1.5\n5/16”\n12S\nHMFS06-M16CF\nM16 x 1.5\n3/8”\n8S\nHMFS06-M18CF\nM18 x 1.5\n3/8”\n10S\nHMFS06-M20CF\nM20 x 1.5\n3/8”\n12S\nHMFS06-M22CF\nM22 x 1.5\n3/8”\n14S\nHMFS08-M22CF\nM22 x 1.5\n1/2”\n14S\nHMFS08-M24CF\nM24 x 1.5\n1/2”\n16S\nHMFS08-M30CF\nM30 x 2\n1/2”\n20S\nHMFS10-M24CF\nM24 x 1.5\n5/8”\n16S\nHMFS10-M30CF\nM30 x 2\n5/8”\n20S\nHMFS12-M30CF\nM30 x 2\n3/4”\n20S\nHMFS12-M36CF\nM36 x 2\n3/4”\n25S\nHMFS16-M36CF\nM36 x 2\n1”\n25S\nHMFS16-M42CF\nM42 x 2\n1”\n30S\nHMFS20-M52CF\nM52 x 2\n1.1/4”\n38S\nHMFS24-M52CF\nM52 x 2\n1.1/2”\n38S\n90¡ Metric Female 24¡ Cone Seat, \nS Series, DIN 3865\n0309\nThread\nTo Suit\nHose ID\nSize\nHMFS0390-M16CF\nM16 x 1.5\n3/16”\n8S\nHMFS0490-M14CF\nM14 x 1.5\n1/4”\n6S\nHMFS0490-M16CF\nM16 x 1.5\n1/4”\n8S\nHMFS0490-M18CF\nM18 x 1.5\n1/4”\n10S\nHMFS0490-M20CF\nM20 x 1.5\n1/4”\n12S\nHMFS0590-M18CF\nM18 x 1.5\n5/16”\n10S\nHMFS0590-M20CF\nM20 x 1.5\n5/16”\n12S\nHMFS0690-M18CF\nM18 x 1.5\n3/8”\n10S\nHMFS0690-M20CF\nM20 x 1.5\n3/8”\n12S\nHMFS0690-M22CF\nM22 x 1.5\n3/8”\n14S\nHMFS0890-M24CF\nM24 x 1.5\n1/2”\n16S\nHMFS1090-M24CF\nM24 x 1.5\n5/8”\n16S\nHMFS1090-M30CF\nM30 x 2\n5/8”\n20S\nHMFS1290-M30CF\nM30 x 2\n3/4”\n20S\nHMFS1290-M36CF\nM36 x 2\n3/4”\n25S\nHMFS1690-M36CF\nM36 x 2\n1”\n25S\nHMFS1690-M42CF\nM42 x 2\n1”\n30S\nHMFS2090-M52CF\nM52 x 2\n1.1/4”\n38S\nHMFS2490-M52CF\nM52 x 2\n1.1/2”\n38S\nStandpipe, \nS Series\n0309\nOD mm\nTo Suit Hose ID\nHSSP04-10CF\n10\n1/4”\nHSSP06-14CF\n14\n3/8”\nHSSP08-16CF\n16\n1/2”\nHSSP10-20CF\n20\n5/8”\nHSSP12-20CF\n20\n3/4”\nHSSP12-25CF\n25\n3/4”\nHSSP16-25CF\n25\n1”\nHSSP16-30CF\n30\n1”\nHSSP20-38CF\n38\n1.1/4”\n45¡ Metric Female 24¡ Cone Seat, \nS Series, DIN 3865\n0309\nThread\nTo Suit\nHose ID\nSize\nHMFS03-45-M16CF\nM16 x 1.5\n3/16”\n8S\nHMFS04-45-M14CF\nM14 x 1.5\n1/4”\n6S\nHMFS04-45-M16CF\nM16 x 1.5\n1/4”\n8S\nHMFS04-45-M18CF\nM18 x 1.5\n1/4”\n10S\nHMFS05-45-M18CF\nM18 x 1.5\n5/16”\n10S\nHMFS05-45-M20CF\nM20 x 1.5\n5/16”\n12S\nHMFS06-45-M20CF\nM20 x 1.5\n3/8”\n12S\nHMFS06-45-M22CF\nM22 x 1.5\n3/8”\n14S\nHMFS08-45-M24CF\nM24 x 1.5\n1/2”\n16S\nHMFS10-45-M30CF\nM30 x 2\n5/8”\n20S\nHMFS12-45-M30CF\nM30 x 2\n3/4”\n20S\nHMFS12-45-M36CF\nM36 x 2\n3/4”\n25S\nHMFS16-45-M36CF\nM36 x 2\n1”\n25S\nHMFS16-45-M42CF\nM42 x 2\n1”\n30S\nHMFS20-45-M52CF\nM52 x 2\n1.1/4”\n38S\nHMFS24-45-M52CF\nM52 x 2\n1.1/2”\n38S\n90¡ Standpipe, \nS Series\n0309\nOD mm\nTo Suit Hose ID\nHSSP04-90-08CF\n8\n1/4”\nHSSP04-90-10CF\n10\n1/4”\nHSSP06-90-14CF\n14\n3/8”\nHSSP08-90-16CF\n16\n1/2”\nHSSP10-90-20CF\n20\n5/8”\nHSSP12-90-20CF\n20\n3/4”\nHSSP12-90-25CF\n25\n3/4”\nHSSP16-90-25CF\n25\n1”\nHSSP16-90-30CF\n30\n1”\n45¡ Standpipe, S Series\n0309\nOD mm\nTo Suit Hose ID\nHSSP04-45-08CF\n8\n1/4”\nHSSP04-45-10CF\n10\n1/4”\nHSSP04-45-12CF\n12\n1/4”\nHSSP05-45-12CF\n12\n5/16”\nHSSP06-45-14CF\n14\n3/8”\nHSSP08-45-14CF\n14\n1/2”\nHSSP08-45-16CF\n16\n1/2”\nHSSP10-45-16CF\n16\n5/8”\nHSSP10-45-20CF\n20\n5/8”\nHSSP12-45-20CF\n20\n3/4”\nHSSP12-45-25CF\n25\n3/4”\nHSSP16-45-25CF\n25\n1”\nHSSP16-45-30CF\n30\n1”\nHSSP16-45-38CF\n38\n1”\nHSSP20-45-30CF\n30\n1.1/4”\nHSSP20-45-38CF\n38\n1.1/4”\nHYDRAULIC HOSE CONNECTORS'
# #         ``````````````````````
      
# #         Output:
# #         ``````````````````````                
# #         index: [0], table_header or descriptor: [Metric Male 24 Cone Seat,S Series, DIN 386], column_names: [0309, thread, to_suit_house_id, size, header]
# #         index: [1], table_header or descriptor: [Standpipe,S Series], column_names: [0309, od_mm, to_suit_house_id]    
# #         ``````````````````````  
                  
# #         NOTE: Some columns have sub columns under them. Simply append the sub columns to the main column and return them as a unit. 
        
# #         Now provide output in line with the example format for the following: 
           
# #           ``````````````````````````
# #           User's request: {user_text}
# #           ``````````````````````````

# #           ``````````````````````````
# #           Table to extract data from: {table_headers[0]}
# #           ``````````````````````````

# #           ``````````````````````````
# #           Text document: {extracted_text}
# #           ``````````````````````````
# #         """






# # def get_image_informations(image_path: str) -> dict:
# #    vision_chain = load_image_chain | image_model | parser
# #    return vision_chain.invoke({'image_path': f'{image_path}', 
# #                                'prompt': vision_prompt})

# # result = get_image_informations('C:/Users/derri/Desktop/pic of 0012.png')

In [51]:
import modules.llm
importlib.reload(modules.llm)
from modules.llm import vision_llm_parser
# use user input to determine table of interst
variable_col_data = vision_llm_parser(user_text=user_text,
                                   text_input= extracted_text,
                                   columns=input_fields,
                                    table_to_target= table_headers[0],
                                    base64_image=base64_image,
                                    open_api_key=open_api_key)
variable_col_data

'``````````````````````````\n\n[farm_name: Melbourne Farm |-| Melbourne Farm |-| Melbourne Farm |-| Melbourne Farm |-| Melbourne Farm],\n\n[organic_items: Apple |-| Orange |-| Watermelon |-| Mango |-| Peach],\n\n[price_kg: $5.00 |-| $1.99 |-| $1.69 |-| $9.56 |-| $2.99],\n\n[quantity_kg: 1 |-| 2 |-| 3 |-| 2 |-| 1],\n\n[subtotal: $5.00 |-| $3.98 |-| $5.07 |-| $19.12 |-| $2.99]\n\n``````````````````````````'

In [52]:
import pandas as pd
text = variable_col_data
pattern = r"\[([^\]:]+):([^]]+)\]"
matches = re.findall(pattern, text, flags=re.DOTALL)

data = {}
max_len = 0

# 2) Split each matched value on "|-|"
for key, val in matches:
    items = [item.replace("***", "").strip()  # optionally remove "***"
             for item in val.split("|-|")]
    data[key.strip()] = items
    max_len = max(max_len, len(items))

# 3) Build a DataFrame, ensuring all columns have the same length
df = pd.DataFrame({
    col: values + [None]*(max_len - len(values))  # pad with None if needed
    for col, values in data.items()
})

df



,farm_name,organic_items,price_kg,quantity_kg,subtotal
0,Melbourne Farm,Apple,$5.00,1,$5.00
1,Melbourne Farm,Orange,$1.99,2,$3.98
2,Melbourne Farm,Watermelon,$1.69,3,$5.07
3,Melbourne Farm,Mango,$9.56,2,$19.12
4,Melbourne Farm,Peach,$2.99,1,$2.99


In [53]:
1/0

ZeroDivisionError: division by zero

In [ ]:
# import re
# import pandas as pd

# text = llm_table_data

# pattern = r"\[(\w+):([^]]+)\]"
# matches = re.findall(pattern, text)

# data = {}
# for col_name, raw_values in matches:
#     values = [v.strip() for v in raw_values.split("|-|")]
#     data[col_name] = values

# # --- Fix or pad mismatched lengths -----------------------------
# max_len = max(len(vals) for vals in data.values())
# for col_name, vals in data.items():
#     if len(vals) < max_len:
#         # Append None (or "" or some placeholder) to match the maximum length
#         vals += [None] * (max_len - len(vals))

# df = pd.DataFrame(data)
# df

In [ ]:
# columsn may be changeed due to llm being overhweled.

In [ ]:
# import re
# import pandas as pd

# text = llm_table_data

# # 1) Regex pattern to capture things of the form:
# #    [column_name: val1 |-| val2 |-| ...]
# pattern = r"\[(\w+):([^]]+)\]"

# # 2) Find all matches; each match is (column_name, "val1 |-| val2 |-| ...")
# matches = re.findall(pattern, text)

# # 3) Build a dictionary { column_name: [list_of_values, ...] }
# data = {}
# for col_name, raw_values in matches:
#     # Remove any extra newlines/spaces and split on "|-|"
#     values = [v.strip() for v in raw_values.split("|-|")]
#     data[col_name] = values

# # 4) Create the DataFrame
# df = pd.DataFrame(data)



In [ ]:
# import importlib
# import modules.llm
# importlib.reload(modules.llm)


# from modules.llm import table_extraction


# table_output = table_extraction(pattern_data = pattern_desc_from_image, text_data =extracted_text, model = "gpt-4o", openai_client=openai_client)
# table_output

In [ ]:
results

[{'index': 0,
  'table_header': 'Box SPM.1, Table 1: Description and relationship of scenarios and modelled pathways considered across AR6 Working Group reports',
  'column_names': ['category_in_wgiii',
   'category_description',
   'ghg_emissions_scenarios_sspx-y_in_wgi_and_wgii',
   'rcpy_in_wgi_and_wgii']}]

In [ ]:
results[0]['column_names']

['category_in_wgiii',
 'category_description',
 'ghg_emissions_scenarios_sspx_y_in_wgi_wgii',
 'rcpy_in_wgi_wgii']

In [ ]:
input_fields = list(set(results[0]['column_names']))
input_fields

['category_in_wgiii',
 'ghg_emissions_scenarios_sspx_y_in_wgi_wgii',
 'category_description',
 'rcpy_in_wgi_wgii']

In [ ]:
from pydantic import create_model
import json
import pandas as pd 

import importlib
import modules.llm
importlib.reload(modules.llm)
from modules.llm import llm_parser


import importlib
import modules.llm
importlib.reload(modules.llm)
from modules.llm import variable_extractor_llm


# Could targeting tables then extracting from each one by one and then concating be the best way to go ?
# how do i identify tables

# If user provided then

#  pattern = pattern_output,
# If no user provided then get a default template

# pattern_output = pattern_description_llm(text_input=extracted_text, base64_image = base64_image, open_api_key= open_api_key )


# var_list =  variable_extractor_llm(user_text=text_1, text_data=extracted_text, openai_client=openai_client, model='o3-mini') # gpt-4o
table_index = 0

input_fields = sorted(set(results[table_index]['column_names'])) # in case of dups


# Assume these are the field names coming from the front end
# input_fields = list(set(var_list))

# input_fields.append('table_number')  # To differential tables
# print(input_fields)

#   Build a dictionary defining the field names and their types.
#  I will assume each field required a list of strings.
field_definitions = {field: (list[str], ...) for field in input_fields}

# Dynamically create a new Pydantic model 
dynamic_structured_output_class = create_model('structured_output', **field_definitions)

target_table = results[table_index]['table_header']
print(f"Target table {target_table}")
print(f"Columns {input_fields} ")

completion = llm_parser(user_text = user_text, 
                        target_table=target_table,
                        target_variables= input_fields,
                        openai_client = openai_client,
                        pattern=parsing_rules,
                        text_document=extracted_text,
                        response_format=dynamic_structured_output_class,
                        model='gpt-4o') # o3-mini, gpt-4o

out  = completion.choices[0].message.content

data = json.loads(out)

# To ensure all columns are of the same length
# Determine the maximum length among the arrays
max_len = max(len(lst) for lst in data.values())

# Pad each list to have the same length
for key, lst in data.items():
    if len(lst) < max_len:
        lst.extend([None] * (max_len - len(lst)))

# Create a DataFrame from the dictionary
df = pd.DataFrame(data)

df # ensure columsn returnd are ascii clean white spaces

#error handling try and except 
# Perhaps retry when JSON error happens
# Factorise table numbers. Sometimes i can choose 2 with no other tables present

Target table Description and relationship of scenarios and modelled pathways considered across AR6 Working Group reports
Columns ['category_description', 'category_in_wgiii', 'ghg_emissions_scenarios_sspx_y_in_wgi_wgii', 'rcpy_in_wgi_wgii'] 


2025-02-10 23:55:19,368 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


,category_description,category_in_wgiii,ghg_emissions_scenarios_sspx_y_in_wgi_wgii,rcpy_in_wgi_wgii
0,limit warming to 1.5°C (>50%) with no or limit...,C1,Very low (SSP1-1.9),RCP2.6
1,return warming to 1.5°C (>50%) after a high ov...,C2,Low (SSP1-2.6),
2,limit warming to 2°C (>67%),C3,,
3,limit warming to 2°C (>50%),C4,,
4,limit warming to 2.5°C (>50%),C5,,
5,limit warming to 3°C (>50%),C6,Intermediate (SSP2-4.5),RCP 4.5
6,limit warming to 4°C (>50%),C7,High (SSP3-7.0),RCP 8.5
7,exceed warming of 4°C (>50%),C8,Very high (SSP5-8.5),


In [ ]:
target_table

'Description and relationship of scenarios and modelled pathways considered across AR6 Working Group reports'

In [ ]:
1/0

ZeroDivisionError: division by zero

In [ ]:
1/0

In [ ]:
!pip install pdfplumber

In [ ]:
import pdfplumber
import pandas as pd

def extract_tables_from_pdf(pdf_path, output_csv=None):
    tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            extracted_tables = page.extract_tables()
            for table in extracted_tables:
                tables.append(pd.DataFrame(table))
    
    # Combine tables into one DataFrame if needed
    if tables:
        df = pd.concat(tables, ignore_index=True)
        if output_csv:
            df.to_csv(output_csv, index=False)
        return df
    else:
        return None

# Example usage

output_csv = "output.csv"  # Optional: specify a CSV file to save the extracted table
df = extract_tables_from_pdf(pdf_path=pdf_path, output_csv=output_csv)

if df is not None:
    print(df.head())  # Display first few rows
else:
    print("No tables found in the PDF.")


In [ ]:
from img2table.document import PDF

doc = PDF(src=pdf_path, 
          pages=None,
          detect_rotation=False,
          pdf_text_extraction=True)

In [ ]:
from img2table.ocr import TesseractOCR
from img2table.document import Image
import pytesseract

# Instantiation of OCR
ocr = TesseractOCR(n_threads=1, lang="eng")

# Instantiation of document, either an image or a PDF
# doc = Image(pdf_path)

# Table extraction
extracted_tables = doc.extract_tables(ocr=ocr,
                                      implicit_rows=False,
                                      implicit_columns=False,
                                      borderless_tables=False,
                                      min_confidence=50)

In [ ]:
df.drop_duplicates(subset='id_information')

In [ ]:
extracted_text

In [ ]:
dynamic_structured_output_class

In [ ]:
pdf_path

In [ ]:
print(dynamic_structured_output_class.__annotations__)


In [ ]:
data

In [ ]:
# Create a DataFrame from the dictionary
df = pd.DataFrame(data)

df

In [ ]:
extracted_text_list

In [ ]:
out  = completion.choices[0].message.content
out

In [ ]:
# Parse the JSON string into a dictionary
import json
import pandas as pd 

# Display the DataFrame
print(df)

In [ ]:
prompt_variable_extractor = """ 

 You are a skilled data extraction and parsing specialist with expertise in analysing text documents and extracting structured information, these could be from a varity of sources including PDFs and many others. 
          Your job is to extract data for the provided variable(s) from a text document in line with the user's request and return them in JSON format where each value is associated with each other.  

 
          ''''''''''''''''
          User's request: " I would like to extract the Order No and Model from the pdf document. The order number looks something like this (1051042839) and the model No is an mixture of numbers and letters and looks like this for instance NLOLC1
          I would also like to extract the heading which is usually uppercased and above the order and model number. 
          ''''''''''''''''
           
          Target variable(s): [model_number, order_number]
           
text document: ['\n\n--- Page 3 ---\n\n\tSTEEL ENCLOSED CONTACTORS\nOrder No.\nModel No.\nAC1 Heating & \nGeneral Load\nFluorescent \nMercury\nHalogen\nLED\nNo. of Poles\nDimensions HxWxD\nSTEEL ENCLOSED CONTACTORS\n1051042857\nNLCONM25/4\n25A\n16A\n13A\n25A\n4 + N\n184 x 184 x 149mm\n1051042858\nNLCONM45/4\n40A\n27A\n22A\n40A\n4 + N\n184 x 184 x 149mm\n1051042859\nNLCONM63/3N\n60A\n40A\n32A\n60A\n3 + N\n305 x 285 x 159mm\n•\tManufactured in accordance with EN60947-4-1 \n•\tIP55 steel enclosure\n•\tStandard coil voltage: 230V 50/60Hz\n•\tAuxiliary contacts can be fitted\n•\tColour RAL7035 light grey\n•\tKnockouts top and bottom\n•\tAll units are suitable for two wire control i.e. time \nswitch, thermostat, sensor or other remote controls\n•\tSupplied complete with neutral terminals\nNLCONM100/3N\n\tDIRECT ON LINE, REVERSING & \nSTAR DELTA STARTERS\n•\tManufactured in accordance with EN60947-4-1 \n•\tIP55 steel enclosure\n•\tColour RAL7035 light grey\n•\tSurface mounting\n•\tStandard coil voltages: 230V or 400V 50/60Hz\n•\tBuilt in start/stop and reset push button\n•\tHand reset thermal overloads\n•\tOverload to be ordered separately\n•\tStainless steel pozidrive fixing screws for lid\n•\tKnockouts top and bottom\n•\tInternal components DIN rail mounted\nOrder No.\nModel No.\nMax Rating\nControl Voltage\nDimensions H x W x D\nType\nDIRECT ON LINE & REVERSING STARTERS\n1051042849\nNLSTM27C\n7.5kW\n230V\n174 x 104 x 134mm\nDirect on Line\n£0.00\n1051042850\nNLSTM47C\n7.5kW\n400V\n174 x 104 x 134mm\nDirect on Line\n£0.00\n1051042856\nNLRVM45C\n5.5kW\n400V\n184 x 184 x 149mm\nReversing\n£0.00\n1051042851\nNLSTM25CSW\n5.5kW\n230V\n184 x 184 x 149mm\nDOL With Isolator\n£0.00\n1051042852\nNLSTM45CSW\n5.5kW\n400V\n184 x 184 x 149mm\nDOL With Isolator\n£0.00\n1051042854\nNLSDM415C\n15kW\n400V\n184 x 284 x 149mm\nStar Delta\n£0.00\nNLSTM27C\nStock items delivered FREE anywhere in UK\n15 3\n\n',
 '\n']

Output:

[
  {
    "heading": "STEEL ENCLOSED CONTACTORS",
    "order_number": "1051042857",
    "model_number": "NLCONM25/4"
  },
  {
    "heading": "STEEL ENCLOSED CONTACTORS",
    "order_number": "1051042858",
    "model_number": "NLCONM45/4"
  },
  {
    "heading": "STEEL ENCLOSED CONTACTORS",
    "order_number": "1051042859",
    "model_number": "NLCONM63/3N"
  },
  {
    "heading": "DIRECT ON LINE, REVERSING & STAR DELTA STARTERS",
    "order_number": "1051042849",
    "model_number": "NLSTM27C"
  },
  {
    "heading": "DIRECT ON LINE, REVERSING & STAR DELTA STARTERS",
    "order_number": "1051042850",
    "model_number": "NLSTM47C"
  },
  {
    "heading": "DIRECT ON LINE, REVERSING & STAR DELTA STARTERS",
    "order_number": "1051042856",
    "model_number": "NLRVM45C"
  },
  {
    "heading": "DIRECT ON LINE, REVERSING & STAR DELTA STARTERS",
    "order_number": "1051042851",
    "model_number": "NLSTM25CSW"
  },
  {
    "heading": "DIRECT ON LINE, REVERSING & STAR DELTA STARTERS",
    "order_number": "1051042852",
    "model_number": "NLSTM45CSW"
  },
  {
    "heading": "DIRECT ON LINE, REVERSING & STAR DELTA STARTERS",
    "order_number": "1051042854",
    "model_number": "NLSDM415C"
  }
]


"""
